# Download

# Kafka

In [ ]:
curl -o kafka_2.13-3.4.0.tgz https://www.apache.org/dyn/closer.cgi?path=/kafka/3.4.0/kafka_2.13-3.4.0.tgz
tar -xzf kafka_2.13-3.4.0.tgz
cd kafka_2.13-3.4.0

bin/zookeeper-server-start.sh config/zookeeper.properties
bin/kafka-server-start.sh config/server.properties

In [ ]:
docker run -it --network host -e DYNAMIC_CONFIG_ENABLED=true docker.repos.balad.ir/provectuslabs/kafka-ui

# Airflow

In [ ]:
export AIRFLOW_HOME=~/airflow

AIRFLOW_VERSION=2.6.0
PYTHON_VERSION="$(python --version | cut -d " " -f 2 | cut -d "." -f 1-2)"
CONSTRAINT_URL="https://raw.githubusercontent.com/apache/airflow/constraints-${AIRFLOW_VERSION}/constraints-${PYTHON_VERSION}.txt"

pip install "apache-airflow==${AIRFLOW_VERSION}" --constraint "${CONSTRAINT_URL}"


In [ ]:
AIRFLOW__WEBSERVER__WEB_SERVER_PORT=8081 AIRFLOW__CORE__LOAD_EXAMPLES=False

# Test Kafka

In [14]:
!pip install confluent-kafka

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [16]:
from confluent_kafka import Producer
import socket

conf = {'bootstrap.servers': "localhost:9092",
        'client.id': socket.gethostname()}

producer = Producer(conf)

In [18]:
producer.produce(topic="testi", value="salam2")
producer.flush()

0

In [20]:
from confluent_kafka import Consumer

conf = {'bootstrap.servers': 'localhost:9092',
        'group.id': "foo_group",
        'auto.offset.reset': 'earliest'}

consumer = Consumer(conf)

In [21]:
consumer.subscribe(["testi"])

In [28]:
msg = consumer.poll(timeout=1.0)

In [29]:
msg.value()

AttributeError: 'NoneType' object has no attribute 'value'

%4|1683380593.892|MAXPOLL|rdkafka#consumer-3| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 100ms (adjust max.poll.interval.ms for long-running message processing): leaving group


# Stream

In [59]:
import pandas as pd

df = pd.read_csv("/home/divar/university/MLOps/Project/data/flickr30k_text.csv/results.csv", sep="|")
df.head()

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [ ]:
from confluent_kafka import Producer
import socket

conf = {'bootstrap.servers': "localhost:9092",
        'client.id': socket.gethostname()}

producer = Producer(conf)

In [64]:
df.count()

image_name         158915
 comment_number    158915
 comment           158915
dtype: int64

In [65]:
import json

for i, row in df.iterrows():
    try:
        producer.produce(topic="english_only_data", value=json.dumps({
            "dataset": "flickr30k",
            "image_id": row["image_name"],
            "english_sentence": row[" comment"][1:],
        }))
    except Exception as e:
        print(row)
        raise e
    if i % 1000 == 0:
        producer.flush()

producer.flush()

0

# Translate

In [12]:
!pip install googletrans==3.1.0a0

  Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.16.3
    Uninstalling httpcore-0.16.3:
      Successfully uninstalled httpcore-0.16.3
  Attempting uninstall: httpx
    Found existing installation: httpx 0.23.3
    Uninstalling httpx-0.23.3:
      Successfully uninstalled httpx-0.23.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-proje

In [3]:
from googletrans import Translator
translator = Translator()

def translate(english_sentence):
    return translator.translate(english_sentence, src='en', dest='fa').text

translate("The weather is great here!")

'هوا اینجا عالیه!'

In [68]:
def translate_and_produce(msg_value):
    data = json.loads(msg_value)
    data["farsi_sentence"] = translate(data["english_sentence"])
    producer.produce(topic="translated_data", value=json.dumps(data))

In [69]:
from confluent_kafka import Consumer

conf = {'bootstrap.servers': 'localhost:9092',
        'group.id': "translate",
        'auto.offset.reset': 'earliest'}

consumer = Consumer(conf)
consumer.subscribe(["english_only_data"])

from confluent_kafka import KafkaError, KafkaException

while True:
    msg = consumer.poll(timeout=1.0)

    if msg is None or msg.error():
        if msg and msg.error().code() != KafkaError._PARTITION_EOF:
            raise KafkaException(msg.error())
        break
    
    translate_and_produce(msg.value())

ReadTimeout: The read operation timed out

%4|1683383627.785|MAXPOLL|rdkafka#consumer-4| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 236ms (adjust max.poll.interval.ms for long-running message processing): leaving group


# Score Translation

In [ ]:
!pip install sentence-transformers

In [9]:
from sentence_transformers import SentenceTransformer
from numpy import dot
from numpy.linalg import norm

translation_scoring_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

def calculate_sentence_similarity(sent1, sent2):
    """
    Returns cosine similarity of embeddings of to given sentences.
    """
    emb1, emb2 = translation_scoring_model.encode([sent1, sent2])
    cos_sim = dot(emb1, emb2) / (norm(emb1) * norm(emb2))
    return cos_sim

In [72]:
def validate_translation_and_produce(msg_value):
    data = json.loads(msg_value)
    
    score = calculate_sentence_similarity(data["english_sentence"], data["farsi_sentence"])
    if score >= 0.7:
        data["translation_score"] = str(score)
        producer.produce(topic="validated_translation_data", value=json.dumps(data))

In [10]:
calculate_sentence_similarity(
    "The weather is great here!",
    "هوا اینجا عالیه!",
)

0.8394125

In [13]:
calculate_sentence_similarity(
    "The weather is great here!",
    "سلام خوبی!",
)

0.3100414

In [76]:
from confluent_kafka import Consumer

conf = {'bootstrap.servers': 'localhost:9092',
        'group.id': "validate_translation",
        'auto.offset.reset': 'earliest'}

consumer = Consumer(conf)
consumer.subscribe(["translated_data"])

from confluent_kafka import KafkaError, KafkaException

while True:
    msg = consumer.poll(timeout=1.0)

    if msg is None or msg.error():
        if msg and msg.error().code() != KafkaError._PARTITION_EOF:
            raise KafkaException(msg.error())
        break

    validate_translation_and_produce(msg.value())

# Aggregate & Save

In [2]:
import json

from confluent_kafka import Consumer
import time

conf = {'bootstrap.servers': 'localhost:9092',
        'group.id': f"save_data_{int(time.time())}",
        'auto.offset.reset': 'earliest'}

consumer = Consumer(conf)
consumer.subscribe(["validated_translation_data"])

from confluent_kafka import KafkaError, KafkaException

data = []
while True:
    msg = consumer.poll(timeout=1.0)

    if msg is None or msg.error():
        if msg and msg.error().code() != KafkaError._PARTITION_EOF:
            raise KafkaException(msg.error())
        break

    data.append(json.loads(msg.value()))

In [8]:
import pandas as pd

df = pd.DataFrame.from_dict(data)

In [9]:
df.head()

,dataset,image_id,english_sentence,farsi_sentence,translation_score
0,flickr30k,1000092795.jpg,"Two young , White males are outside near many ...",دو نر جوان سفید پوست بیرون در نزدیکی بسیاری از...,0.8116357
1,flickr30k,1000092795.jpg,Two men in green shirts are standing in a yard .,دو مرد با پیراهن سبز در حیاط ایستاده اند.,0.7840364
2,flickr30k,1000092795.jpg,A man in a blue shirt standing in a garden .,مردی با پیراهن آبی در باغ ایستاده است.,0.90311915
3,flickr30k,1000092795.jpg,Two friends enjoy time spent together .,دو دوست از زمان با هم بودن لذت می برند.,0.90404236
4,flickr30k,10002456.jpg,Several men in hard hats are operating a giant...,چند مرد با کلاه سخت در حال کار با یک سیستم قرق...,0.90194845


In [ ]:
df.to_csv("/home/divar/university/MLOps/Project/data/cleaned_flickr30k.csv")

# Crawl Divar

In [15]:
import requests

response = requests.get("https://api.divar.ir/v8/posts/wY9VsSUG", timeout=5)
post_data = response.json()

In [23]:
print(json.dumps(post_data, indent=4, ensure_ascii=False))

{
    "data": {
        "nabz_product_id": null,
        "business_card_slug": null,
        "seo": {
            "title": "لپ تاپ قدرتمند hp با رم ۱۲ گیگ|رایانه همراه|ارومیه|دیوار",
            "description": "لپ تاپ قدرتمند hp با رم ۱۲ گیگ|رایانه همراه|ارومیه|خرید و فروش ✅دستگاه شامل گارانتی رایگان(مهلت تست) و فاکتور رسمی میباشد\n✅تضمین قیمت و کیفیت در سراسر استان\n✅امکان فروش اقساطی(تا ۳ ماه) بدون سود و بهره *فقط کافیست دسته چک تونو بردارین و بیایین فروشگاه*\n✅یک ماه خدمات پس از فروش (بدون چون و چرا)\n________________________\nلپ تاپ قوی hp\n⬅️cpu : core i5 3310m\n⬅️ram: 12 GB\n⬅️hard: 320 GB\n⬅️vga :4 GB intel\n⬅️دارای HDMIو وبکم و usb3\n⬅️فول پورت و نگهداری باتری بسیار فوق العاده\n⬅️تمیز در حد صفر\n⬅️از این ارزان تر با این امکانات نداریم\n⬅️هدیه : کیف به خریدار محترم\n_________________________\nمشاوره رایگان\nتخفیف ویژه برای خرید نقدی\n در فروشگاه ما آداپتور.باتری.led.dvd با قیمت های بسیار مناسب به فروش میرسد\nارسال سریع به سراسر ایران\nآدرس جهت خرید حضوری:\nارومیه،خیابان امام ،پا

In [25]:
post_data["widgets"]["header"]["title"]

'لپ تاپ قدرتمند hp با رم ۱۲ گیگ'

In [22]:
post_data["widgets"]["images"]

['https://s100.divarcdn.com/static/pictures/1675863954/wY9VsSUG.webp',
 'https://s100.divarcdn.com/static/pictures/1675863954/wY9VsSUG.1.webp',
 'https://s100.divarcdn.com/static/pictures/1675863954/wY9VsSUG.2.webp',
 'https://s100.divarcdn.com/static/pictures/1675863954/wY9VsSUG.3.webp',
 'https://s100.divarcdn.com/static/pictures/1675863954/wY9VsSUG.4.webp']

In [27]:
post_token = "wY9VsSUG"

for i, image_url in enumerate(post_data["widgets"]["images"]):
    with open(f'/home/divar/university/MLOps/Project/data/divar/{post_token}-{i}.jpg', 'wb') as handle:
        response = requests.get(image_url, stream=True)
        if not response.ok:
            print(response)
        for block in response.iter_content(1024):
            if not block:
                break
            handle.write(block)


In [28]:
with open(f'/home/divar/university/MLOps/Project/data/divar/{post_token}.txt', 'w') as file:
    file.write(post_data["widgets"]["header"]["title"])
